# Recommending movies: retrieval

Real world Recommender systems are often made up of following steps - 

* `Retrieval` :   Selecting few thousands of possible candidates to recommend from a set of millions. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.

* `Ranking` : Drill down thousand possible candidates to few hundreds

* `Post-Ranking` : Further refine the candidates to few dozens - Might be helpful in case user is logged from mobile device where screen real estate is limited

In this notebook, we will go over Retrieval stage of the Recommender System. Retrieval models are often composed of two sub-models:

* A `query model` computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
* A `candidate model` computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

## Dataset

The Movielens dataset is a classic dataset from the GroupLens research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of `implicit feedback`, where users' watches tell us which things they prefer to see and which they'd rather not see.

2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of `explicit feedback`: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.


In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

## Imports

In [ ]:
! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann
! pip install tfds-nightly

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

## EDA

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
# Let's look at features from ratings datasets
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

In [ ]:
# And features from movies datasets
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

In [ ]:
# Extract relevant features for our task - User ID and Movie Title
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

### Train-Test Split

We'll go for 80-20 split

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
# Let's also figure out unique user ids and movie titles present in the data.

# This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. 
# To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: 
# this allows us to look up the corresponding embeddings in our embedding tables.

In [ ]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

## Model Implementation

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower

In [ ]:
# The first step is to decide on the dimensionality of the query and candidate representations:
embedding_dimension = 32

# Higher values will correspond to models that may be more accurate, 
# but will also be slower to fit and more prone to overfitting.

In [ ]:
# The second is to define the model itself. 
# Here, we're going to use Keras preprocessing layers to first convert user ids to integers,
# and then convert those to user embeddings via an Embedding layer. 
# Note that we use the list of unique user ids we computed earlier as a vocabulary

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### The candidate tower

In [ ]:
# Same approach as the query tower

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])